

{% include toc title="In This Lesson" icon="file-text" %}



<div class='notice--success' markdown="1">



## <i class="fa fa-graduation-cap" aria-hidden="true"></i> Learning Objectives



After completing this tutorial, you will be able to:



* Define Canopy Height Model (CHM), Digital Elevation Model (DEM) and Digital Surface Model (DSM).

* Describe the key differences between the **CHM**, **DEM**, **DSM**.

* Derive a **CHM** in R using raster math.



## <i class="fa fa-check-square-o fa-2" aria-hidden="true"></i> What you need



You need `R` and `RStudio` to complete this tutorial. Also you should have

an `earth-analytics` directory setup on your computer with a `/data`

directory with it.



* [How to Setup R / RStudio](/course-materials/earth-analytics/week-1/setup-r-rstudio/)

* [Setup your working directory](/course-materials/earth-analytics/week-1/setup-working-directory/)

* [Intro to the R & RStudio Interface](/course-materials/earth-analytics/week-1/intro-to-r-and-rstudio)



### R Libraries to Install:



* **raster:** `install.packages("raster")`

* **rgdal:** `install.packages("rgdal")`



If you have not already downloaded the week 3 data, please do so now.



[<i class="fa fa-download" aria-hidden="true"></i> Download Week 3 Data (~250 MB)](https://ndownloader.figshare.com/files/7446715){:data-proofer-ignore='' .btn }



</div>



As we discussed in the previous lesson, LiDAR or **Li**ght **D**etection **a**nd

**R**anging is an active remote sensing system that can be used to measure

vegetation height across wide areas. If the data are discrete return, Lidar point

clouds are most commonly derived data product from a lidar system. However, often

people work with lidar data in raster format given it's smaller in size and

thus easier to work with. In this lesson, we will import and work with

3 of the most common lidar derived data products in `R`:



1. **Digital Terrain Model (or DTM):** ground elevation.

2. **Digital Surface Model (or DSM):** top of the surface (imagine draping a sheet over the canopy of a forest

3. **Canopy Height Model (CHM):** the elevation of the Earth's surface - and it sometimes also called a DEM or digital elevation model.



## 3 Important Lidar Data Products: CHM, DEM, DSM



<figure>

   <a href="{{ site.url }}/images/course-materials/earth-analytics/week-3/lidarTree-height.png">

   <img src="{{ site.url }}/images/course-materials/earth-analytics/week-3/lidarTree-height.png" alt="Lidar derived DSM, DTM and CHM."></a>

   <figcaption>Digital Surface Model (DSM), Digital Elevation Models (DEM) and

   the Canopy Height Model (CHM) are the most common raster format lidar

   derived data products. One way to derive a CHM is to take

   the difference between the digital surface model (DSM, tops of trees, buildings

   and other objects) and the Digital Terrain Model (DTM, ground level). The CHM

   represents the actual height of trees, buildings, etc. with the influence of

   ground elevation removed. Graphic: Colin Williams, NEON

   </figcaption>

</figure>





### Digital Elevation Model

In the previous lesson, we opened a digital elevation model. The digital elevation

model (DEM), also known as a digital terrain model (DTM) represents the elevation

of the earth surfacel. The DEM represents the ground - and thus DOES NOT INCLUDE

trees and buildings and other objects.




In [1]:
```{r load-libraries,  message=F, warning=F }

# load libraries

library(raster)

library(rgdal)



# set working directory to ensure R can find the file we wish to import

# setwd("working-dir-path-here")

```




First, let's open and plot the digital elevation model.




In [1]:
```{r dem, fig.cap="digital elevation model plot", warning=F, message=F }

# open raster data

lidar_dem <- raster(x="data/week3/BLDR_LeeHill/pre-flood/lidar/pre_DTM.tif")



# plot raster data

plot(lidar_dem,

     main="Lidar Digital Elevation Model (DEM)")



```




Next, let's open the digital SURFACE model (DSM). The DSM represents the top of

the earth's surface. Thus, it INCLUDES TREES, BUILDINGS and other objects that

sit on the earth.




In [1]:
```{r dsm, fig.cap="digital surface model plot", warning=F, message=F }

# open raster data

lidar_dsm <- raster(x="data/week3/BLDR_LeeHill/pre-flood/lidar/pre_DSM.tif")



# plot raster data

plot(lidar_dsm,

     main="Lidar Digital Surface Model (DSM)")

```




## Canopy Height Model



The canopy height model (CHM) represents the HEIGHT of the trees. This is not

an elevation value, rather it's the height or distance between the ground and the top of the

trees. Some canopy height models also include buildings so you need to look closely

are your data to make sure it was properly cleaned before assuming it represents

all trees!



### Calculate difference between two rasters



There are different ways to calculate a CHM. One easy way is to subtract the

DEM from the DSM.



DSM - DEM = CHM



This math gives you the residual value or difference between the top of the

earth surface and the ground which should be the heights of the trees (and buildings

if the data haven't been "cleaned").




In [1]:
```{r chm, fig.cap="canopy height model plot", warning=F, message=F }

# open raster data

lidar_chm <- lidar_dsm - lidar_dem



# plot raster data

plot(lidar_chm,

     main="Lidar Canopy Height Model (CHM)")

```




## Plots Using Breaks



Sometimes a gradient of colors is useful to represent a continuous variable.

But other times, it's useful to apply colors to particular ranges of values

in a raster. These ranges may be statistically generated or simply visual.



Let's create breaks in our CHM plot.






In [1]:
```{r chm-breaks, fig.cap="canopy height model breaks", warning=F, message=F}

# plot raster data

plot(lidar_chm,

     breaks = c(0, 2, 10, 20, 30),

     main="Lidar Canopy Height Model",

     col=c("white","brown","springgreen","darkgreen"))



```




## Export a raster



When can export a raster file in R using the `write.raster()` function. Let's

export the canopy height model that we just created to our data folder. We will

create a new directory called "outputs" within the week 3 directory. This structure

allows us to keep things organized, separating our outputs from the data we downloaded.



NOTE: you can use the code below to check for and create an outputs directory.

OR, you can create the directory yourself using the finder (MAC) or windows

explorer.






In [1]:
```{r export-raster, warning=F, message=FALSE}

# check to see if an output directory exists

dir.exists("data/week3/outputs")



# if the output directory doesn't exist, create it

if (dir.exists("data/week3/outputs")) {

  print("the directory exists!")

  } else {

    # if the directory doesn't exist, create it

    # recursive tells R to create the entire directory path (data/week3/outputs)

    dir.create("data/week3/outputs", recursive=TRUE)

  }



# export CHM object to new GeotIFF

writeRaster(lidar_chm, "data/week3/outputs/lidar_chm.tiff",

            format="GTiff",  # output format = GeoTIFF

            overwrite=TRUE) # CAUTION: if this is true, it will overwrite an existing file





```
